In [1]:
import pandas as pd
import os 
import torch 
import numpy as np
import torch.nn as nn
import torch.optim as optim 

In [3]:
for dirname, _, filenames in os.walk('/archive/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))